In [38]:
#importing dependencies
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
data = pd.read_excel('top_clgs_new.xlsx')
data.head()

,College Name,College City,State,Approvals,Placement,Cut_Off,List_of_Courses
0,IIM Bangalore - Indian Institute of Management,Bangalore,Karnataka,"['AICTE', 'UGC']","[{'Batch': 'Number of Companies Participated',...","[{'College': 'IIM Ahmedabad', 'Minimum Expecte...",NaN
1,IIT Madras - Indian Institute of Technology - ...,Chennai,Tamil Nadu,['AICTE'],"[{'Batch': 'Number of Companies Participated',...","[{('Courses', 'Courses'): 'B.Tech Computer Sci...","[{0: 'Program', 1: 'Eligibility', 2: 'IIT Madr..."
2,Universal Business School - [UBS],Karjat,Maharashtra,"['AICTE', 'NBA', 'AIU']",[{'Particulars': 'Highest International packag...,"[{'Name of the Exam': 'CAT', 'Cutoff': '70%ile...",NaN
3,IIT Delhi - Indian Institute of Technology [IITD],New Delhi,Delhi NCR,[],"[{'Particulars': 'PPOs Offered', 'Placement St...","[{('B.Tech Specialisation', 'B.Tech Specialisa...","[{('IIT Delhi Courses & Fees 2023', 'Program')..."
4,IIT Bombay - Indian Institute of Technology - ...,Mumbai,Maharashtra,"['AICTE', 'UGC']",[{'Particulars': 'No. of Companies offered Job...,"[{('Courses', 'Courses'): 'B.Tech Computer Sci...",NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10010 entries, 0 to 10009
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   College Name     10010 non-null  object
 1   College City     10010 non-null  object
 2   State            10010 non-null  object
 3   Approvals        10010 non-null  object
 4   Placement        1086 non-null   object
 5   Cut_Off          4136 non-null   object
 6   List_of_Courses  749 non-null    object
dtypes: object(7)
memory usage: 547.5+ KB


In [6]:
data.isnull().sum()

College Name          0
College City          0
State                 0
Approvals             0
Placement          8924
Cut_Off            5874
List_of_Courses    9261
dtype: int64

In [7]:
data.columns

Index(['College Name', 'College City', 'State', 'Approvals', 'Placement',
       'Cut_Off', 'List_of_Courses'],
      dtype='object')

In [62]:
data.rename(columns={'College City':'City'},inplace=True)

In [8]:
data.shape

(10010, 7)

In [22]:
unique_state_names = data['State'].value_counts().sort_index(ascending=True)

In [23]:
len(unique_state_names)

33

In [24]:
unique_state_names

Andaman and Nicobar Islands       3
Andhra Pradesh                  422
Arunachal Pradesh                11
Assam                           150
Bihar                           143
Chandigarh                       54
Chhattisgarh                    115
Delhi NCR                       324
Goa                              34
Gujarat                         503
Haryana                         365
Himachal Pradesh                 83
Jammu and Kashmir                43
Jharkhand                        87
Karnataka                       839
Kerala                          577
Madhya Pradesh                  439
Maharashtra                    1432
Manipur                          11
Meghalaya                        21
Mizoram                           9
Nagaland                         16
Orissa                          285
Puducherry                       40
Punjab                          360
Rajasthan                       365
Sikkim                           12
Tamil Nadu                  

In [30]:
unique_city_names = data['College City'].unique()

In [31]:
len(unique_city_names)

849

In [32]:
unique_city_names

array(['Bangalore', 'Chennai', 'Karjat', 'New Delhi', 'Mumbai',
       'Kharagpur', 'Kanpur', 'Roorkee', 'Pilani', 'Guwahati',
       'Ahmedabad', 'Chandigarh', 'Bikaner', 'Hyderabad', 'Kolkata',
       'Lucknow', 'Vellore', 'Varanasi', 'Tiruchirappalli', 'Manipal',
       'Pondicherry', 'Coimbatore', 'Guntur', 'Indore', 'Dhanbad',
       'Bhopal', 'Rourkela', 'Pune', 'Navi Mumbai', 'Mohali', 'Ludhiana',
       'Bhubaneswar', 'Howrah', 'Mangalore', 'Bareilly', 'Aligarh',
       'Jamshedpur', 'Karnal', 'Mandi', 'Kozhikode', 'Thiruvananthapuram',
       'Aurangabad', 'Vijayawada', 'Surathkal', 'Anand', 'Mysore',
       'Jodhpur', 'Nagercoil', 'Patna', 'North Goa', 'Gandhinagar',
       'Nagpur', 'Madurai', 'Pantnagar', 'Dharwad', 'Patiala',
       'Ernakulam', 'Kochi', 'Panji', 'Gurgaon', 'Thane', 'Visakhapatnam',
       'Secunderabad', 'Udaipur', 'Noida', 'Chidambaram', 'Imphal',
       'Tirupati', 'Udagamandalam', 'Moradabad', 'Gwalior', 'Warangal',
       'Kottayam', 'Raipur', 'Shillo

In [33]:
city_count = data['College City'].value_counts().sort_index(ascending=True)

In [34]:
city_count

Abohar           3
Adilabad         1
Adoni            1
Adoor            1
Agartala         9
                ..
Yamuna Nagar    21
Yanam            2
Yavatmal         7
Ziro             1
Zunhebotto       2
Name: College City, Length: 849, dtype: int64

In [35]:
selecting_features = ['College Name', 'College City', 'State', 'List_of_Courses'];
for feature in selecting_features:
    data[feature] = data[feature].fillna('')

In [36]:
combine_feat = data['College Name']+data['College City']+data['State']+data['List_of_Courses']

In [37]:
print(combine_feat)

0        IIM Bangalore - Indian Institute of Management...
1        IIT Madras - Indian Institute of Technology - ...
2        Universal Business School - [UBS]KarjatMaharas...
3        IIT Delhi - Indian Institute of Technology [II...
4        IIT Bombay - Indian Institute of Technology - ...
                               ...                        
10005    Katuri Medical College and HospitalGunturAndhr...
10006    Kempegowda Institute of Medical Sciences - [KI...
10007    Kempegowda Institute of Physiotherapy - [KIPT]...
10008    Khaja Bandanawaz Institute of Medical Sciences...
10009    Government Kilpauk Medical College - [GKMC]Che...
Length: 10010, dtype: object


In [39]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combine_feat)


In [40]:
print(feature_vectors)

  (0, 8757)	0.5441093212233986
  (0, 10278)	0.10506897416224462
  (0, 6791)	0.15487440284755968
  (0, 6721)	0.3551557839528093
  (0, 1919)	0.5209528988756006
  (0, 6575)	0.5209528988756006
  (1, 131)	0.05433107111255238
  (1, 6437)	0.06331951754767864
  (1, 6418)	0.0583462572735465
  (1, 4043)	0.048627530395243874
  (1, 9973)	0.08707893861006645
  (1, 14646)	0.04342647645115011
  (1, 503)	0.04961874497346596
  (1, 8427)	0.04309779770569692
  (1, 9047)	0.04698035720806546
  (1, 9098)	0.04817895321507176
  (1, 10794)	0.10778346379650176
  (1, 1)	0.04464548469933383
  (1, 0)	0.05543708650685299
  (1, 12004)	0.05252791574015948
  (1, 4516)	0.06041034678098512
  (1, 5881)	0.03403326468336915
  (1, 15725)	0.05337299699941437
  (1, 3849)	0.032610004955179774
  (1, 4795)	0.03909792616723982
  :	:
  (10006, 9135)	0.3055176918527456
  (10006, 12531)	0.3064520339221887
  (10006, 10278)	0.09933033335525909
  (10006, 6791)	0.14641549692195208
  (10007, 7796)	0.6029589486276338
  (10007, 7704)	0.577

In [41]:
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.02089472 0.         ... 0.03118099 0.02579777 0.        ]
 [0.02089472 1.         0.         ... 0.00727892 0.0216296  0.0166454 ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.03118099 0.00727892 0.         ... 1.         0.02296608 0.        ]
 [0.02579777 0.0216296  0.         ... 0.02296608 1.         0.06648943]
 [0.         0.0166454  0.         ... 0.         0.06648943 1.        ]]


In [42]:
choice_state = input('Enter the state where you want your college')
choice_city = input('Enter the city of your choice')


In [43]:
state_name = data['State'].tolist()
city_name = data['College City'].tolist()
college_name = data['College Name'].tolist()

In [44]:
find_close_match = difflib.get_close_matches(choice_state, state_name)
print(find_close_match)

['Chhattisgarh', 'Chhattisgarh', 'Chhattisgarh']


In [45]:
find_close_match_city = difflib.get_close_matches(choice_city, city_name)
print(find_close_match_city)

['Bhilai', 'Bhilai', 'Bhilai']


In [46]:
close_match_state = find_close_match[0]
close_match_city = find_close_match_city[0]


In [53]:
name_of_college = data[data.State == close_match_state]['College Name'].values
print(name_of_college)

['Indian Institute of Management - [IIM]'
 'Hidayatullah National Law University - [HNLU]'
 'National Institute of Technology - [NIT]'
 'Dr. C.V. Raman University - [CVRU]'
 'Government Jamuna Prasad Verma Post Graduate Arts and Commerce College'
 'Indira Gandhi Krishi Vishwavidyalaya - [IGKV]'
 'Guru Ghasidas Vishwavidyalaya - [GGU]'
 'Dignity College of Architecture - [DCA]' 'ITM University'
 'Shaheed Gundadhur College of Agriculture and Research - [SGCARS]'
 'Mahant Laxminarayan Das College'
 'Pt. Ravishankar Shukla University - [PRSU]'
 'RMD College of Agriculture and Research Station'
 'Kushabhau Thakre Patrakarita Avam Jansanchar Vishwavidyalaya'
 'Chhattisgarh Kamdhenu Vishwavidyalaya - [CGKV]' 'MATS University'
 'IIT Bhilai - Indian Institute of Technology - [IITB]'
 'Government Mahaprabhu Vallabhacharya Post Graduate College'
 'Amity University'
 'International Institute of Information Technology - [IIIT] Naya Raipur'
 'Bhilai Institute of Technology - [BIT]'
 'Bhilai Institut

In [63]:
name_of_college = data[data.City == close_match_city]['College Name'].values

In [64]:
print(name_of_college)

['IIT Bhilai - Indian Institute of Technology - [IITB]'
 'Christian College of Engineering and Technology - [CCET]'
 'Rungta College of Engineering and Technology - [RCET]'
 'Krishna Engineering College - [KEC]'
 'RSR Rungta College of Engineering and Technology - [RSRRCET]'
 'Shri Shankaracharya Group of Institutions - [SSGI]'
 'Rungta Group of Institutions Bhilai Campus'
 'Rungta Engineering College - [REC]'
 'Chhattisgarh institute of management and technology - [CIMT]'
 'Parthivi College of Engineering and Management'
 'Garv Institute of Management and Technology - [GIMT]'
 'GD Rungta College of Engineering and Technology - [GDRCET]'
 'Swami Shri Swaroopanand Saraswati Mahavidyalaya - [SSSSMV]'
 'Rungta College of Dental Sciences and Research - [RCDSR]']


In [68]:
choice_city = input('Enter the city where you want your college')
find_close_match = difflib.get_close_matches(choice_city, city_name)
close_match_city = find_close_match[0]
name_of_college = data[data.City == close_match_city]['College Name'].values

print('User\'s choice of preference for City is : ',choice_city)

print('Colleges recommended for user : \n')

i = 0
for movie in name_of_college:
  index = movie[0]
  name_of_college = data[data.City == close_match_city]['College Name'].values[i]
  if (i<15):
    print(i+1, '.',name_of_college)
    i+=1

User's choice of preference for City is :  Hyderabad
Colleges recommended for user : 

1 . IIT Hyderabad - Indian Institute of Technology - [IITH]
2 . IIT Hyderabad - Indian Institute of Technology - [IITH]
3 . NALSAR University of Law - [NALSAR]
4 . University of Hyderabad - [UOH]
5 . Institute Of Hotel Management Catering Technology & Applied Nutrition- [IHMH ]
6 . National Institute of Fashion Technology - [NIFT]
7 . National Institute of Pharmaceutical Education and Research - [NIPER]
8 . Indian School of Business - [ISB]
9 . St Ann's College for Women
10 . International Institute of Information Technology - [IIIT]
11 . Jawaharlal Nehru Technological University - [JNTUH]
12 . University college of Law, Osmania University
13 . ICFAI Business School - [IBS]
14 . Jawaharlal Nehru Architecture and Fine Arts University - [JNAFAU]
15 . ICFAI Law School
